In [7]:
#Import Dependencies

from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np

from time import sleep
from random import randint

In [8]:
# Sets a path to Google Chrome
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/joescuteri/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


In [11]:
URL = ['https://twitchtracker.com/games', 'https://twitchtracker.com/games?page=2', 'https://twitchtracker.com/games?page=3', 'https://twitchtracker.com/games?page=4', 'https://twitchtracker.com/games?page=5']
tags = []
games = []
cleanedviews = []
views = []

for url in range(0,5):      
    browser.visit(URL[url])
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    href_tags = soup.find_all(href=True)
    raw_games = href_tags[40:60]
    
    for x in raw_games:
        tags.append(str(x))
    
    for element in tags:
        if element not in games:
            front = element.split(">")[1]
            back = front.split("<")[0]
            games.append(back)
        
    viewhtml = soup.find_all('div', class_="ri-value")
    
    for x in viewhtml:
        cleanedviews.append(str(x))
    

    for element in cleanedviews:
        if element not in views:
            front = element.split(">")[2]
            back = front.split("<")[0]
            views.append(back)
            
    print(f'Page Done!')
    
    sleep(randint(5,10))

Page Done!
Page Done!
Page Done!
Page Done!
Page Done!


In [12]:
# Upload to DataFrame 
df = pd.DataFrame(games, columns = ['GAME'])
df['VIEWS'] = views
df

,GAME,VIEWS
0,League of Legends,388K
1,Grand Theft Auto V,287K
2,VALORANT,214K
3,Fortnite,121K
4,Apex Legends,104K
...,...,...
295,Food &amp; Drink,3.1K
296,"Animals, Aquariums, and Zoos",3.1K
297,Farming Simulator 22,3.0K
298,Mario Kart 8 Deluxe,3.0K


In [13]:
# Change VIEWS column to integers

df.VIEWS = (df.VIEWS.replace(r'[KM]+$', '', regex=True).astype(float) * df.VIEWS.str.extract(r'[\d\.]+([KM]+)', expand=False)
             .fillna(1)
             .replace(['K','M'], [10**3, 10**6]).astype(int))

In [14]:
avg_views = df['VIEWS'].mean()
median_views = df['VIEWS'].median()

In [15]:
df

,GAME,VIEWS
0,League of Legends,388000.0
1,Grand Theft Auto V,287000.0
2,VALORANT,214000.0
3,Fortnite,121000.0
4,Apex Legends,104000.0
...,...,...
295,Food &amp; Drink,3100.0
296,"Animals, Aquariums, and Zoos",3100.0
297,Farming Simulator 22,3000.0
298,Mario Kart 8 Deluxe,3000.0


In [16]:
df.drop(df.tail(1).index,inplace=True)

In [17]:
df

,GAME,VIEWS
0,League of Legends,388000.0
1,Grand Theft Auto V,287000.0
2,VALORANT,214000.0
3,Fortnite,121000.0
4,Apex Legends,104000.0
...,...,...
294,Five Nights at Freddy's: Security Breach,3200.0
295,Food &amp; Drink,3100.0
296,"Animals, Aquariums, and Zoos",3100.0
297,Farming Simulator 22,3000.0


In [18]:
# Convert DataFrame into Dictionary
twitch = pd.Series(df.VIEWS.values,index=df.GAME).to_dict()

In [19]:
twitch

{'League of Legends': 388000.0,
 'Grand Theft Auto V': 287000.0,
 'VALORANT': 214000.0,
 'Fortnite': 121000.0,
 'Apex Legends': 104000.0,
 'Rust': 88900.0,
 'Minecraft': 88000.0,
 'Escape from Tarkov': 72200.0,
 'Dota 2': 68900.0,
 'Call of Duty: Warzone': 65000.0,
 'Counter-Strike: Global Offensive': 58000.0,
 'FIFA 22': 51200.0,
 'God of War': 49600.0,
 'Slots': 40100.0,
 'Teamfight Tactics': 34700.0,
 'Music': 33700.0,
 'Monster Hunter Rise': 33000.0,
 'Hearthstone': 32100.0,
 'Dead by Daylight': 30300.0,
 '2': 30000.0,
 'Special Events': 28000.0,
 'PUBG: BATTLEGROUNDS': 24200.0,
 'Yu-Gi-Oh! Master Duel': 23300.0,
 'Rocket League': 22400.0,
 'Sports': 21000.0,
 'Genshin Impact': 19900.0,
 'LOST ARK': 18000.0,
 'ASMR': 17700.0,
 "Tom Clancy's Rainbow Six Extraction": 17400.0,
 'Chess': 15800.0,
 'Art': 13400.0,
 'Overwatch': 13000.0,
 'Old School RuneScape': 12900.0,
 'Clash Royale': 12500.0,
 'Talk Shows &amp; Podcasts': 11800.0,
 'Final Fantasy XIV Online': 11300.0,
 'Virtual Casin

# Migrating it to MongoDB

In [20]:
# connecting to mongodb
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# declare the database
db = client.twitchviews_db
collection = db.items

In [21]:
# Push into MongoDB
collection.insert_one(twitch)